In [ ]:
import torch
import numpy as np
import torchvision
import os
import cv2
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def read_files(X, Y, path, ans):
  files = os.listdir(path)
  for name in files:
    img = cv2.imread(path + '/' + name, 0)
    if img.shape != 0:
      img = cv2.resize(img, (256, 256))
      vect = img.reshape(1, 256 ** 2)
      vect = vect / 255.
      X = vect if (X is None) else np.vstack((X, vect)) 
      Y = np.append(Y, ans)
  return X, Y

In [ ]:
path = "/content/drive/MyDrive/DL_Egor_Konyagin_HSE/lesson1_dataset"
X = None
Y = np.array([])
X, Y = read_files(X, Y, path + "/logloss_0", 0)
X, Y = read_files(X, Y, path + "/logloss_1", 1)

In [ ]:
class BoxesDataset(Dataset):
    def __init__(self, X, Y):
        self.x = torch.from_numpy(X)
        self.y = torch.from_numpy(Y)

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [382]:
dataset = BoxesDataset(X, Y)
lengt = [round(0.65*len(dataset)), round(0.15*len(dataset)), round(0.20*len(dataset))]
train_set, eval_set, test_set = torch.utils.data.random_split(dataset, lengt)

train_loader = DataLoader(dataset=train_set, batch_size=8, drop_last=True, shuffle=True)
eval_loader = DataLoader(dataset=eval_set, shuffle=True)
test_loader = DataLoader(dataset=test_set, shuffle=True)

In [383]:
for x, y in train_loader:
    print(x.shape, y.shape)

torch.Size([8, 65536]) torch.Size([8])
torch.Size([8, 65536]) torch.Size([8])
torch.Size([8, 65536]) torch.Size([8])
torch.Size([8, 65536]) torch.Size([8])
torch.Size([8, 65536]) torch.Size([8])


In [384]:
model = nn.Sequential(
    nn.Linear(65536, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(1024, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(1024, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(1024, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(1024, 2),
    nn.Softmax()
)

In [385]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [386]:
model.cuda()

Sequential(
  (0): Linear(in_features=65536, out_features=1024, bias=True)
  (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Dropout(p=0.3, inplace=False)
  (3): ReLU()
  (4): Linear(in_features=1024, out_features=1024, bias=True)
  (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Dropout(p=0.3, inplace=False)
  (7): ReLU()
  (8): Linear(in_features=1024, out_features=1024, bias=True)
  (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): Dropout(p=0.3, inplace=False)
  (11): ReLU()
  (12): Linear(in_features=1024, out_features=1024, bias=True)
  (13): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): Dropout(p=0.3, inplace=False)
  (15): ReLU()
  (16): Linear(in_features=1024, out_features=2, bias=True)
  (17): Softmax(dim=None)
)

In [387]:
epochs = 100
for i in range(epochs):
  for j,(x,y) in enumerate(train_loader):
    model.train()
    optimizer.zero_grad()
    x = x.cuda()
    y = y.cuda()
    y_pred = model(x.float())
    loss = criterion(y_pred, y.long())
    accuracy = (torch.argmax(y_pred.detach().cpu(), 1) == y.cpu()).sum()/len(y)
    loss.backward()
    print('epochs {0} iter {1} loss {2} accuracy {3}'.format(i, j, loss, accuracy))
    optimizer.step()
  with torch.no_grad():
    model.eval()
    for j, (x, y) in enumerate(eval_loader):
      x = x.cuda()
      y = y.cuda()
      y_pred = model(x.float())
      accuracy = (torch.argmax(y_pred.detach().cpu(), 1) == y.cpu()).sum()/len(y)
      print('epochs {0} iter {1}  accuracy {2}'.format(i, j, accuracy))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epochs 0 iter 0 loss 0.6860905885696411 accuracy 0.625
epochs 0 iter 1 loss 0.5731626152992249 accuracy 0.75
epochs 0 iter 2 loss 0.5275797247886658 accuracy 0.875
epochs 0 iter 3 loss 0.5025300979614258 accuracy 0.75
epochs 0 iter 4 loss 0.5204092264175415 accuracy 0.75
epochs 0 iter 0  accuracy 1.0
epochs 0 iter 1  accuracy 0.0
epochs 0 iter 2  accuracy 1.0
epochs 0 iter 3  accuracy 0.0
epochs 0 iter 4  accuracy 1.0
epochs 0 iter 5  accuracy 0.0
epochs 0 iter 6  accuracy 1.0
epochs 0 iter 7  accuracy 0.0
epochs 0 iter 8  accuracy 0.0
epochs 0 iter 9  accuracy 0.0
epochs 1 iter 0 loss 0.47157829999923706 accuracy 0.75
epochs 1 iter 1 loss 0.567803680896759 accuracy 0.75
epochs 1 iter 2 loss 0.45492836833000183 accuracy 0.875
epochs 1 iter 3 loss 0.6578868627548218 accuracy 0.625
epochs 1 iter 4 loss 0.5663396716117859 accuracy 0.75
epochs 1 iter 0  accuracy 0.0
epochs 1 iter 1  accuracy 0.0
epochs 1 iter 2  accuracy 0.0
epochs 1 iter 3  accuracy 1.0
epochs 1 iter 4  accuracy 1.0
epoch

In [388]:
model.eval()
preds = []
true = []

for (x,y) in test_loader:
    x = x.cuda()
    y = y.cuda()
    preds.append(int(torch.max(model(x.float()).data, 1)[1]))
    true.append(int(y))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [389]:
print(true)
print(preds)

accuracy_score(true, preds)

[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]
[0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]


0.9230769230769231